# Semantic Layer SQL API - Jupyter Notebook Demo

This notebook demonstrates how to connect to the Semantic Layer Service using the SQL API from Jupyter notebooks.


## 1. Setup and Connection

First, let's install and import the required packages for connecting to the Semantic Layer SQL API.


In [ ]:
# Install required packages if needed
# !pip install psycopg2-binary pandas matplotlib seaborn


In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


In [ ]:
# Connect to Semantic Layer SQL API
conn = psycopg2.connect(
    host="localhost",
    port=5433,
    database="semantic_layer",
    user="jupyter_user"
)

print("✅ Connected to Semantic Layer")


## 2. Explore Available Semantic Models

Let's discover what semantic models are available in our semantic layer.


In [ ]:
# List all semantic models
query = """
SELECT 
    schema_name,
    REPLACE(schema_name, 'sem_', '') as model_name
FROM information_schema.schemata 
WHERE schema_name LIKE 'sem_%'
ORDER BY schema_name
"""

models_df = pd.read_sql_query(query, conn)
models_df


In [ ]:
# Get tables for first model
if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    query = f"""
    SELECT 
        table_name,
        table_type,
        (SELECT COUNT(*) 
         FROM information_schema.columns c 
         WHERE c.table_schema = t.table_schema 
         AND c.table_name = t.table_name) as column_count
    FROM information_schema.tables t
    WHERE table_schema = '{schema}'
    ORDER BY table_name
    """
    
    tables_df = pd.read_sql_query(query, conn)
    print(f"Tables in {schema}:")
    tables_df


## 3. Query Semantic Data

Now let's query actual data from our semantic models.


In [ ]:
# Sample data from fact table
if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    query = f"SELECT * FROM {schema}.fact LIMIT 10"
    
    sample_df = pd.read_sql_query(query, conn)
    print(f"Sample data from {schema}.fact:")
    sample_df.head()


## 4. Business Analytics Queries

Let's run some typical business analytics queries. Note: You may need to adjust column names based on your specific semantic model.


In [ ]:
# Example: Time series analysis
# Note: Adjust column names based on your semantic model

time_series_query = f"""
SELECT 
    DATE_TRUNC('month', order_date) as month,
    SUM(revenue) as monthly_revenue,
    COUNT(DISTINCT customer_id) as unique_customers,
    COUNT(*) as transaction_count
FROM {schema}.fact
WHERE order_date >= CURRENT_DATE - INTERVAL '12 months'
GROUP BY 1
ORDER BY 1
"""

try:
    time_series_df = pd.read_sql_query(time_series_query, conn)
    time_series_df['month'] = pd.to_datetime(time_series_df['month'])
    time_series_df.set_index('month', inplace=True)
    
    # Plot revenue trend
    fig, ax = plt.subplots(figsize=(12, 6))
    time_series_df['monthly_revenue'].plot(ax=ax, marker='o', linewidth=2)
    ax.set_title('Monthly Revenue Trend', fontsize=16)
    ax.set_xlabel('Month')
    ax.set_ylabel('Revenue')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Query failed: {e}")
    print("Please adjust column names to match your semantic model")


In [ ]:
# Example: Top categories analysis
# Note: Adjust column names based on your semantic model

category_query = f"""
SELECT 
    category,
    SUM(revenue) as total_revenue,
    COUNT(DISTINCT customer_id) as customers,
    AVG(revenue) as avg_order_value
FROM {schema}.fact
GROUP BY category
ORDER BY total_revenue DESC
LIMIT 10
"""

try:
    category_df = pd.read_sql_query(category_query, conn)
    
    # Create horizontal bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    category_df.plot.barh(x='category', y='total_revenue', ax=ax, color='skyblue')
    ax.set_title('Top Categories by Revenue', fontsize=16)
    ax.set_xlabel('Total Revenue')
    ax.set_ylabel('Category')
    plt.tight_layout()
    plt.show()
    
    # Display the data
    category_df
    
except Exception as e:
    print(f"Query failed: {e}")
    print("Please adjust column names (e.g., 'category') to match your semantic model")


## 5. Advanced Analytics with Pandas

Let's do some more advanced analytics by combining SQL queries with pandas operations.


In [ ]:
# Example: Cohort Analysis
# Get customer first purchase date and subsequent behavior

cohort_query = f"""
WITH customer_cohorts AS (
    SELECT 
        customer_id,
        DATE_TRUNC('month', MIN(order_date)) as cohort_month,
        DATE_TRUNC('month', order_date) as order_month,
        SUM(revenue) as revenue
    FROM {schema}.fact
    GROUP BY customer_id, DATE_TRUNC('month', order_date)
)
SELECT 
    cohort_month,
    order_month,
    COUNT(DISTINCT customer_id) as customers,
    SUM(revenue) as total_revenue
FROM customer_cohorts
GROUP BY cohort_month, order_month
ORDER BY cohort_month, order_month
"""

try:
    cohort_df = pd.read_sql_query(cohort_query, conn)
    
    # Calculate months since first purchase
    cohort_df['cohort_month'] = pd.to_datetime(cohort_df['cohort_month'])
    cohort_df['order_month'] = pd.to_datetime(cohort_df['order_month'])
    cohort_df['months_since_cohort'] = (
        (cohort_df['order_month'] - cohort_df['cohort_month']) / pd.Timedelta(days=30)
    ).round().astype(int)
    
    # Create cohort retention matrix
    cohort_pivot = cohort_df.pivot_table(
        index='cohort_month',
        columns='months_since_cohort',
        values='customers',
        aggfunc='sum'
    )
    
    # Calculate retention rates
    cohort_retention = cohort_pivot.divide(cohort_pivot[0], axis=0) * 100
    
    # Plot retention heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(cohort_retention.iloc[:6, :6], 
                annot=True, 
                fmt='.1f', 
                cmap='YlOrRd', 
                cbar_kws={'label': 'Retention %'})
    plt.title('Customer Retention by Cohort', fontsize=16)
    plt.xlabel('Months Since First Purchase')
    plt.ylabel('Cohort Month')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Cohort analysis failed: {e}")
    print("This example requires customer_id and order_date columns")


## 6. Working with Metric Views

The Semantic Layer may provide pre-calculated metric views for common analytics needs.


In [ ]:
# Check for metric views
if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    view_query = f"""
    SELECT 
        table_name as view_name,
        table_type
    FROM information_schema.tables
    WHERE table_schema = '{schema}'
    AND table_type = 'VIEW'
    ORDER BY table_name
    """
    
    views_df = pd.read_sql_query(view_query, conn)
    
    if not views_df.empty:
        print(f"Available metric views in {schema}:")
        for view in views_df['view_name']:
            print(f"  - {view}")
            
        # Query a metric view
        metric_view = views_df.iloc[0]['view_name']
        sample_query = f"SELECT * FROM {schema}.{metric_view} LIMIT 5"
        
        print(f"\nSample from {metric_view}:")
        pd.read_sql_query(sample_query, conn)
    else:
        print("No metric views found in this semantic model")


## 7. Exploring Column Metadata

Let's explore the columns and their semantic types to better understand our data model.


In [ ]:
# Get column information for fact table
if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    columns_query = f"""
    SELECT 
        column_name,
        data_type,
        is_nullable,
        column_default,
        -- Extract semantic type from column comment if available
        CASE 
            WHEN col_description(pgc.oid, a.attnum) LIKE '%dimension%' THEN 'dimension'
            WHEN col_description(pgc.oid, a.attnum) LIKE '%measure%' THEN 'measure'
            WHEN col_description(pgc.oid, a.attnum) LIKE '%entity%' THEN 'entity'
            ELSE 'unknown'
        END as semantic_type
    FROM information_schema.columns c
    LEFT JOIN pg_catalog.pg_attribute a ON a.attname = c.column_name
    LEFT JOIN pg_catalog.pg_class pgc ON pgc.relname = c.table_name
    WHERE c.table_schema = '{schema}'
    AND c.table_name = 'fact'
    AND a.attnum > 0
    AND NOT a.attisdropped
    ORDER BY c.ordinal_position
    """
    
    columns_df = pd.read_sql_query(columns_query, conn)
    
    print(f"Columns in {schema}.fact:")
    print("-" * 60)
    
    # Group by semantic type
    for sem_type in ['dimension', 'measure', 'entity', 'unknown']:
        type_cols = columns_df[columns_df['semantic_type'] == sem_type]
        if not type_cols.empty:
            print(f"\n{sem_type.upper()}S:")
            for _, col in type_cols.iterrows():
                print(f"  - {col['column_name']} ({col['data_type']})")


## 8. Performance Tips and Best Practices

When working with the Semantic Layer SQL API, here are some tips for optimal performance.


In [ ]:
# Example: Using EXPLAIN to understand query performance
if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    # Create a test query
    test_query = f"""
    SELECT 
        DATE_TRUNC('month', order_date) as month,
        SUM(revenue) as total_revenue
    FROM {schema}.fact
    WHERE order_date >= CURRENT_DATE - INTERVAL '12 months'
    GROUP BY 1
    """
    
    # Get query plan
    explain_query = f"EXPLAIN (ANALYZE FALSE, FORMAT JSON) {test_query}"
    
    try:
        explain_df = pd.read_sql_query(explain_query, conn)
        plan = explain_df.iloc[0, 0][0]['Plan']
        
        print("Query Execution Plan:")
        print("-" * 60)
        print(f"Node Type: {plan.get('Node Type')}")
        print(f"Strategy: {plan.get('Strategy', 'N/A')}")
        print(f"Startup Cost: {plan.get('Startup Cost', 'N/A')}")
        print(f"Total Cost: {plan.get('Total Cost', 'N/A')}")
        print(f"Plan Rows: {plan.get('Plan Rows', 'N/A')}")
        
    except Exception as e:
        print(f"Could not get query plan: {e}")

# Performance best practices
print("\n📊 Performance Best Practices:")
print("-" * 60)
print("1. Use date filters to limit data scanned")
print("2. Aggregate at the database level, not in pandas")
print("3. Select only needed columns")
print("4. Use proper indexes (handled by semantic layer)")
print("5. Batch multiple queries when possible")


## 9. Error Handling and Debugging

Proper error handling is important when working with database connections.


In [ ]:
# Example: Robust query execution with error handling
def safe_query(connection, query, params=None):
    """Execute a query with proper error handling."""
    try:
        # Create a new cursor for this query
        with connection.cursor() as cursor:
            cursor.execute(query, params)
            
            # For SELECT queries, fetch results
            if cursor.description:
                columns = [desc[0] for desc in cursor.description]
                results = cursor.fetchall()
                return pd.DataFrame(results, columns=columns)
            else:
                # For INSERT/UPDATE/DELETE
                connection.commit()
                return f"Query executed successfully. Rows affected: {cursor.rowcount}"
                
    except psycopg2.Error as e:
        # Rollback on error
        connection.rollback()
        print(f"Database error: {e.pgerror}")
        print(f"Error code: {e.pgcode}")
        return None
    except Exception as e:
        print(f"Unexpected error: {type(e).__name__}: {e}")
        return None

# Test with an invalid query
invalid_query = "SELECT * FROM non_existent_table"
result = safe_query(conn, invalid_query)

# Test with a valid query
if not models_df.empty:
    valid_query = f"SELECT COUNT(*) as row_count FROM {models_df.iloc[0]['schema_name']}.fact"
    result = safe_query(conn, valid_query)
    if result is not None:
        print(f"✅ Valid query result: {result.iloc[0]['row_count']} rows")


## 10. Advanced Use Cases

Here are some advanced analytics examples you can perform with the Semantic Layer.


In [ ]:
# Example: Customer Lifetime Value Analysis
# This combines multiple analytical techniques

if not models_df.empty:
    schema = models_df.iloc[0]['schema_name']
    
    clv_query = f"""
    WITH customer_metrics AS (
        SELECT 
            customer_id,
            MIN(order_date) as first_purchase_date,
            MAX(order_date) as last_purchase_date,
            COUNT(DISTINCT order_date) as purchase_count,
            SUM(revenue) as total_revenue,
            AVG(revenue) as avg_order_value,
            -- Calculate days between first and last purchase
            EXTRACT(DAY FROM MAX(order_date) - MIN(order_date)) as customer_lifespan_days
        FROM {schema}.fact
        GROUP BY customer_id
    ),
    customer_segments AS (
        SELECT 
            *,
            CASE 
                WHEN total_revenue >= (SELECT PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY total_revenue) FROM customer_metrics) THEN 'High Value'
                WHEN total_revenue >= (SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY total_revenue) FROM customer_metrics) THEN 'Medium Value'
                ELSE 'Low Value'
            END as value_segment,
            CASE
                WHEN purchase_count >= 5 THEN 'Frequent'
                WHEN purchase_count >= 2 THEN 'Occasional'
                ELSE 'One-time'
            END as frequency_segment
        FROM customer_metrics
    )
    SELECT 
        value_segment,
        frequency_segment,
        COUNT(*) as customer_count,
        AVG(total_revenue) as avg_lifetime_value,
        AVG(purchase_count) as avg_purchases,
        AVG(customer_lifespan_days) as avg_lifespan_days
    FROM customer_segments
    GROUP BY value_segment, frequency_segment
    ORDER BY value_segment, frequency_segment
    """
    
    try:
        clv_df = pd.read_sql_query(clv_query, conn)
        
        # Create a heatmap of customer segments
        pivot_df = clv_df.pivot(index='value_segment', 
                                columns='frequency_segment', 
                                values='customer_count')
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot_df, annot=True, fmt='g', cmap='Blues')
        plt.title('Customer Segmentation Matrix', fontsize=16)
        plt.ylabel('Value Segment')
        plt.xlabel('Frequency Segment')
        plt.tight_layout()
        plt.show()
        
        # Display segment metrics
        print("Customer Segment Analysis:")
        clv_df
        
    except Exception as e:
        print(f"CLV analysis failed: {e}")
        print("This example requires customer_id, order_date, and revenue columns")
